In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from  selenium import webdriver
from  selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
from datetime import datetime
import gzip
from io import BytesIO
import pandas as pd
import xml.etree.ElementTree as ET

In [42]:

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('detach',True)

driver = webdriver.Chrome(options=chrome_options)

driver.get('https://url.publishedprices.co.il/login')

driver.find_element(By.NAME, value='username').send_keys('RamiLevi')

driver.find_element(By.ID, value='login-button').click()

In [43]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [44]:
td_shop = soup.find_all(name="a", class_= "f")


In [51]:
for i,td in enumerate(td_shop):
    if i <5:
        path_name = td.get("href")
        print(path_name)


/file/d/NULLPromo7290058140886-700-202410211200.gz
/file/d/Price7290058140886-001-202410210800.gz
/file/d/Price7290058140886-001-202410210900.gz
/file/d/Price7290058140886-001-202410211000.gz
/file/d/Price7290058140886-001-202410211100.gz


In [ ]:
for i,td in enumerate(td_shop):
    path_name = td.get("title")
    driver.find_element(By.LINK_TEXT,value = path_name).click()
    print(i)

In [6]:
#price all
url_basic = 'https://url.publishedprices.co.il'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('detach',True)

driver = webdriver.Chrome(options=chrome_options)

driver.get('https://url.publishedprices.co.il/login')

driver.find_element(By.NAME, value='username').send_keys('freshmarket')

driver.find_element(By.ID, value='login-button').click()

time.sleep(10)

soup = BeautifulSoup(driver.page_source, "html.parser")
td_shop = soup.find_all(name="a", class_= "f")

df_all = pd.DataFrame()
error = []
for i,td in enumerate(td_shop):
    file_url = url_basic+td.get("href")
    file_name =td.get("title") 




    if file_name[:5]=='Price':
        
        try:            
            date_string = file_name.split('-')[2].replace('.gz','')
            file_date = datetime.strptime(date_string, "%Y%m%d%H%M")
            num_reshet = file_name.split('-')[0].replace('Price','')
            num_snif = file_name.split('-')[1]
            print(i,file_name)
            # print(file_url)
            cookies = driver.get_cookies()

            session = requests.Session()
            for cookie in cookies:
                session.cookies.set(cookie['name'], cookie['value'])

            response = session.get(file_url)

            with gzip.open(BytesIO(response.content), 'rb') as gz_file:
                extracted_content = gz_file.read()
            xml_content = extracted_content.decode('utf-8')

            root = ET.fromstring(xml_content)
            data = []
            for item in root.findall('.//Items/Item'):
                row = {child.tag: child.text for child in item}
                data.append(row)
            df = pd.DataFrame(data)
            df['file_name'] = file_name   
            # print(file_name)
            df['num_reshet'] = num_reshet
            df['num_snif'] = num_snif
            df['file_date'] = file_date
        except Exception as e:
            error.append([file_name,e])
                 
        df_all = pd.concat([df_all,df], ignore_index=True)

driver.quit()


82 Price7290876100000-001-202411010700.gz
83 Price7290876100000-001-202411010800.gz
84 Price7290876100000-001-202411010900.gz
85 Price7290876100000-001-202411011000.gz
86 Price7290876100000-002-202411010600.gz
87 Price7290876100000-002-202411010700.gz
88 Price7290876100000-002-202411010800.gz
89 Price7290876100000-002-202411010900.gz
90 Price7290876100000-002-202411011000.gz
91 Price7290876100000-003-202411010700.gz
92 Price7290876100000-003-202411010800.gz
93 Price7290876100000-004-202411010700.gz
94 Price7290876100000-004-202411010800.gz
95 Price7290876100000-005-202411010700.gz
96 Price7290876100000-006-202411010700.gz
97 Price7290876100000-006-202411010800.gz
98 Price7290876100000-006-202411011000.gz
99 Price7290876100000-007-202411010700.gz
100 Price7290876100000-007-202411010800.gz
101 Price7290876100000-007-202411010900.gz
102 Price7290876100000-007-202411011000.gz
103 Price7290876100000-008-202411010700.gz
104 Price7290876100000-008-202411010800.gz
105 Price7290876100000-009-20

In [15]:
#promo all
url_basic = 'https://url.publishedprices.co.il'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('detach',True)

driver = webdriver.Chrome(options=chrome_options)

driver.get('https://url.publishedprices.co.il/login')

driver.find_element(By.NAME, value='username').send_keys('freshmarket')

driver.find_element(By.ID, value='login-button').click()

time.sleep(10)

soup = BeautifulSoup(driver.page_source, "html.parser")
td_shop = soup.find_all(name="a", class_= "f")

df_all_promo = pd.DataFrame()
error = []
for i,td in enumerate(td_shop):
    file_url = url_basic+td.get("href")
    file_name =td.get("title") 
    if file_name[:5]=='Promo':
        
        try:
            file_url = url_basic+td.get("href")
            file_name =td.get("title") 
            print(i,file_name)
            # print(file_url)
            cookies = driver.get_cookies()

            session = requests.Session()
            for cookie in cookies:
                session.cookies.set(cookie['name'], cookie['value'])

            response = session.get(file_url)

            with gzip.open(BytesIO(response.content), 'rb') as gz_file:
                extracted_content = gz_file.read()
            xml_content = extracted_content.decode('utf-8')

            root = ET.fromstring(xml_content)
            data = []
            for item in root.findall('.//Promotions/Promotion'):
                row = {child.tag: child.text for child in item}
                data.append(row)
            df = pd.DataFrame(data)
            df['file_name'] = file_name   
            # print(file_name)
        except Exception as e:
            error.append([file_name,e])
                 
        df_all_promo = pd.concat([df_all_promo,df], ignore_index=True)

driver.quit()


233 PromoFull7290876100000-001-202410310010.gz
234 PromoFull7290876100000-002-202410310010.gz
235 PromoFull7290876100000-003-202410310010.gz
236 PromoFull7290876100000-004-202410310010.gz
237 PromoFull7290876100000-005-202410310010.gz
238 PromoFull7290876100000-006-202410310010.gz
239 PromoFull7290876100000-007-202410310010.gz
240 PromoFull7290876100000-008-202410310011.gz
241 PromoFull7290876100000-009-202410310010.gz
242 PromoFull7290876100000-009-202410310101.gz
243 PromoFull7290876100000-010-202410310010.gz
244 PromoFull7290876100000-011-202410310010.gz
245 PromoFull7290876100000-012-202410310011.gz
246 PromoFull7290876100000-013-202410310011.gz
247 PromoFull7290876100000-014-202410310010.gz
248 PromoFull7290876100000-016-202410310010.gz
249 PromoFull7290876100000-017-202410310010.gz
250 PromoFull7290876100000-018-202410310010.gz
251 PromoFull7290876100000-019-202410310010.gz
252 PromoFull7290876100000-020-202410310010.gz
253 PromoFull7290876100000-021-202410310010.gz
254 PromoFull

In [ ]:
#one file
url_basic = 'https://url.publishedprices.co.il'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('detach',True)

driver = webdriver.Chrome(options=chrome_options)

driver.get('https://url.publishedprices.co.il/login')

driver.find_element(By.NAME, value='username').send_keys('freshmarket')

driver.find_element(By.ID, value='login-button').click()

time.sleep(10)


file = '/file/d/PriceFull7290876100000-008-202410310010.gz'
file_url = url_basic+file


cookies = driver.get_cookies()

session = requests.Session()
for cookie in cookies:
    session.cookies.set(cookie['name'], cookie['value'])

response = session.get(file_url)

with gzip.open(BytesIO(response.content), 'rb') as gz_file:
    extracted_content = gz_file.read()
xml_content = extracted_content.decode('utf-8')

root = ET.fromstring(xml_content)
data = []
for item in root.findall('.//Items/Item'):
    row = {child.tag: child.text for child in item}
    data.append(row)
    df = pd.DataFrame(data)
    df['file_name'] = file   
        # print(file_name)     

driver.quit()

In [23]:
df[df.ItemCode =='7290000052311']

,PriceUpdateDate,ItemCode,ItemType,ItemName,ManufacturerName,ManufactureCountry,ManufacturerItemDescription,UnitQty,Quantity,UnitOfMeasure,bIsWeighted,QtyInPackage,ItemPrice,UnitOfMeasurePrice,AllowDiscount,ItemStatus,ItemId,file_name
1266,2024-10-28 12:35:11,7290000052311,1,גבינה צהובה עמק לייט,לא ידוע,לא ידוע,גבינה צהובה עמק לייט,לא ידוע,200.00,100 גרם,0,לא ידוע,21.20,0.1060,1,1,126055,/file/d/PriceFull7290876100000-008-20241031001...
2046,2024-10-20 07:23:33,7290000052311,1,גבינה צהובה עמק לייט,לא ידוע,לא ידוע,גבינה צהובה עמק לייט,לא ידוע,200.00,100 גרם,0,לא ידוע,19.90,0.0995,1,1,26115,/file/d/PriceFull7290876100000-008-20241031001...


In [ ]:

from sqlalchemy import create_engine
import pyodbc

#function that export df to sql server
def export_to_SQL(table_name,src,schema='dbo',if_exists='replace'):
    Server = 'LAPTOP-2Q3KRK2D'
    Database = 'MARKET'
    connection_string = f"mssql+pyodbc://@{Server}/{Database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
    engine = create_engine(connection_string)
    src.to_sql(table_name, con=engine, if_exists=if_exists, index=False)


In [9]:
export_to_SQL('freshmarket',df_all)

In [17]:
export_to_SQL('freshmarket_promo',df_all_promo)